# Your Title Here

**Name(s)**: Carter Tran, Anirudh Annabathula

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

In [2]:
import datetime
import pathlib

## Step 1: Introduction

In [3]:
data_path = pathlib.Path('data') / 'Major Power Outage Risks.xlsx'
df = pd.read_excel(data_path, header=5, skiprows=[6], index_col='OBS').drop('variables', axis=1)
df

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,73.27,15.28,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaN,...,59.90,19.90,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,56.65,26.73,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256


In [4]:
df.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

## Step 2: Data Cleaning and Exploratory Data Analysis

In [5]:
def fix_time(time):
    if isinstance(time, datetime.time):
        return pd.Timedelta(hours=time.hour, minutes=time.minute, seconds=time.second)
    return time

tidy_df = (df
           .rename(columns={col: col.replace('U.S.', 'US').replace('.', '_').lower() for col in df.columns})
           .reset_index(drop=True)
           .assign(
               outage_start = lambda df_: pd.to_datetime(df_['outage_start_date'], errors='coerce') + df_['outage_start_time'].apply(fix_time),
               outage_restoration = lambda df_: pd.to_datetime(df_['outage_restoration_date'], errors='coerce') + df_['outage_restoration_time'].apply(fix_time)
           )
           .drop(['outage_start_date', 'outage_start_time', 'outage_restoration_date', 'outage_restoration_time'], axis=1)
)
tidy_df

,year,month,us_state,postal_code,nerc_region,climate_region,anomaly_level,climate_category,cause_category,cause_category_detail,...,popden_urban,popden_uc,popden_rural,areapct_urban,areapct_uc,pct_land,pct_water_tot,pct_water_inland,outage_start,outage_restoration
0,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
1,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
2,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
3,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
4,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1530,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,Coal,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT
1531,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00
1532,2009,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,islanding,NaN,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 11:00:00,2009-08-29 14:01:00


In [6]:
img_dir = pathlib.Path('assets') / 'img'

In [7]:
outages_over_time = tidy_df.set_index('outage_start')['year'].rename('Outage Count').resample("1ME").count()
fig = px.bar(outages_over_time, labels={'value': 'Monthly Outages', 'outage_start': 'Date'}, title='Power Outages over Time', width=700)
fig.update_layout(showlegend=False)
fig.write_image(img_dir / 'outages_over_time.jpg')
fig

## Step 3: Assessment of Missingness

In [8]:
# TODO

## Step 4: Hypothesis Testing

In [9]:
# TODO

## Step 5: Framing a Prediction Problem

In [10]:
# TODO

## Step 6: Baseline Model

In [11]:
# TODO

## Step 7: Final Model

In [12]:
# TODO

## Step 8: Fairness Analysis

In [13]:
# TODO